# SPSS Modelerのタイムスタンプ関連clem関数をPythonで書き換える。


■利用データ<BR>
タイムスタンプ(TS),電力(Power),温度(Temperature),圧力(Pressure),起動時間(Uptime),状態(Status),エラーコード(Outcome)が記録された時系列のセンターデータを使います。毎分のセンサーの値が記録されています。今回はこの中のTSのみを使います。

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/59840a58-0896-227e-fb86-f127ea04b7b6.png)


In [2]:
!wget https://raw.githubusercontent.com/hkwd/200611Modeler2Python/master/data/Cond4n_e.csv

'wget' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


# ①タイムスタンプデータの読み込み
Pythonのタイムスタンプ型の確認

In [3]:
import datetime
import numpy as np
import pandas as pd

#datetime.datetime
dtdt=datetime.datetime(2021, 6, 10, 3, 11, 0)
print(dtdt)
print(type(dtdt))

#numpy.datetime64
npdt=np.datetime64('2021-06-10 03:11:00')
print(npdt)
print(type(npdt))

#pandas._libs.tslibs.timestamps.Timestamp
pddt=pd.to_datetime('2021-06-10 03:11:00')
print(pddt)
print(type(pddt))

dfdt =pd.DataFrame({'datetime':[dtdt],
                   'npdatetime':[npdt],
                   'pddatetime':[pddt]})
print(dfdt.dtypes)
print(type(dfdt['datetime'][0]))
print(type(dfdt['npdatetime'][0]))
print(type(dfdt['pddatetime'][0]))
dfdt

2021-06-10 03:11:00
<class 'datetime.datetime'>
2021-06-10T03:11:00
<class 'numpy.datetime64'>
2021-06-10 03:11:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
datetime      datetime64[ns]
npdatetime    datetime64[ns]
pddatetime    datetime64[ns]
dtype: object
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,datetime,npdatetime,pddatetime
0,2021-06-10 03:11:00,2021-06-10 03:11:00,2021-06-10 03:11:00


csvファイルから読むこむときにはパースしてタイムスタンプ型として読み込むことができます。
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/a8eb965d-56c3-166d-4093-990a381c230e.png)

In [4]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
pd.options.display.max_rows = 10
#print(pd.options.display.max_rows)

import pandas as pd
#df = pd.read_csv('sampletranDEPT4en2019S.csv', parse_dates=[1])
df = pd.read_csv('COND2nts.csv', parse_dates=['TS'])
print(df.dtypes)
print(type(df['TS'][0]))
df

TS             datetime64[ns]
Power                   int64
Temperature             int64
Pressure                int64
Uptime                  int64
Status                  int64
Outcome                 int64
dtype: object
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,TS,Power,Temperature,Pressure,Uptime,Status,Outcome
0,2021-04-01 03:10:00,1079,308,0,3,0,202
1,2021-04-01 03:11:00,1083,308,0,3,0,202
2,2021-04-01 03:12:00,1080,309,0,3,0,202
3,2021-04-01 03:13:00,1080,309,0,3,0,202


# ② 文字列や数値からのタイムスタンプデータ生成



文字列および数値から日付型データ生成
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/34de417c-b214-9c47-15aa-cbc5e0d89689.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/e5b11c2d-183e-b02e-7630-a98029855bda.png)



In [5]:
import datetime
df1=df.copy()
#文字列から日付型データ生成
df1['REF_TS']=pd.to_datetime('2021-04-01 03:11:00')
#数値から日付型データ生成
df1['REF_TS2']=datetime.datetime(2021,4,1,3,11,0)
#pd.to_datetimeでの変換は不要
#df1['REF_TS2']=pd.to_datetime(datetime.datetime(2021,4,1,3,11,0))
print(df1.dtypes)
df1

TS             datetime64[ns]
Power                   int64
Temperature             int64
Pressure                int64
Uptime                  int64
Status                  int64
Outcome                 int64
REF_TS         datetime64[ns]
REF_TS2        datetime64[ns]
dtype: object


,TS,Power,Temperature,Pressure,Uptime,Status,Outcome,REF_TS,REF_TS2
0,2021-04-01 03:10:00,1079,308,0,3,0,202,2021-04-01 03:11:00,2021-04-01 03:11:00
1,2021-04-01 03:11:00,1083,308,0,3,0,202,2021-04-01 03:11:00,2021-04-01 03:11:00
2,2021-04-01 03:12:00,1080,309,0,3,0,202,2021-04-01 03:11:00,2021-04-01 03:11:00
3,2021-04-01 03:13:00,1080,309,0,3,0,202,2021-04-01 03:11:00,2021-04-01 03:11:00


# ③ 現在タイムスタンプの取得
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/3f70309c-7f22-caab-369c-0e831de39f81.png)



In [6]:
import datetime
df2=df.copy()
df2['TODAYTS']=datetime.datetime.now()
#以下でも動作するがマニュアルに見当たらなかった。
#df2['TODAY']=np.datetime64('now')
print(df2.dtypes)
#print(type(df2['TODAY'][0]))
df2

TS             datetime64[ns]
Power                   int64
Temperature             int64
Pressure                int64
Uptime                  int64
Status                  int64
Outcome                 int64
TODAYTS        datetime64[ns]
dtype: object


,TS,Power,Temperature,Pressure,Uptime,Status,Outcome,TODAYTS
0,2021-04-01 03:10:00,1079,308,0,3,0,202,2021-06-21 13:29:22.376219
1,2021-04-01 03:11:00,1083,308,0,3,0,202,2021-06-21 13:29:22.376219
2,2021-04-01 03:12:00,1080,309,0,3,0,202,2021-06-21 13:29:22.376219
3,2021-04-01 03:13:00,1080,309,0,3,0,202,2021-06-21 13:29:22.376219


Pythonで現在時刻・日付・日時を取得 | note.nkmk.me 
<br>https://note.nkmk.me/python-datetime-now-today/
<br>date - Python - Get current time in numpy datetime64 format - Stack Overflow 
<br>https://stackoverflow.com/questions/19461258/python-get-current-time-in-numpy-datetime64-format/44602127

#  ④タイムスタンプの年、月、日、時、分、秒の分解
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/0e387c23-f251-645b-f35d-7c0bc3911bbc.png)


In [7]:
df3=df.copy()
df3['MIN']=df3['TS'].dt.minute
print(df3.dtypes)
df3

TS             datetime64[ns]
Power                   int64
Temperature             int64
Pressure                int64
Uptime                  int64
Status                  int64
Outcome                 int64
MIN                     int64
dtype: object


,TS,Power,Temperature,Pressure,Uptime,Status,Outcome,MIN
0,2021-04-01 03:10:00,1079,308,0,3,0,202,10
1,2021-04-01 03:11:00,1083,308,0,3,0,202,11
2,2021-04-01 03:12:00,1080,309,0,3,0,202,12
3,2021-04-01 03:13:00,1080,309,0,3,0,202,13


pandasで日付・時間の列を処理（文字列変換、年月日抽出など） | note.nkmk.me https://note.nkmk.me/python-pandas-datetime-timestamp/

# ⑤ タイムスタンプの差の計算　
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/c01b01af-6dac-b675-6f9c-562af22e5fce.png)


In [8]:
df4=df.copy()
#文字列から日付型データ生成
df4['REF_TS']=pd.to_datetime('2021-04-01 03:11:00')

df4["TS_DIFF_TD"]=(df4["TS"]-df4["REF_TS"])
print(type((df4["TS_DIFF_TD"])[0]))

#以下は見た目も分かりやすい
df4["TS_DIFF秒"]=(df4["TS"]-df4["REF_TS"]).dt.total_seconds().astype('int')
df4["TS_DIFF分"]=(df4["TS"]-df4["REF_TS"]).dt.total_seconds()/60
df4["TS_DIFF時"]=(df4["TS"]-df4["REF_TS"]).dt.total_seconds()/(60*60)

#以下だとマイナスの時に間違った値が戻る
#df4["TS_DIFF"]=(df4["TS"]-df4["REF_TS"]).dt.seconds

#以下でも計算はできる
#df4["TS_DIFF秒"]=((df4["TS"]-df4["REF_TS"]) /np.timedelta64(1,'s')).astype('int')
#df4["TS_DIFF分"]=((df4["TS"]-df4["REF_TS"]) /np.timedelta64(1,'m'))
#df4["TS_DIFF時"]=((df4["TS"]-df4["REF_TS"]) /np.timedelta64(1,'h'))

print(df4.dtypes)
df4


<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
TS              datetime64[ns]
Power                    int64
Temperature              int64
Pressure                 int64
Uptime                   int64
                    ...       
REF_TS          datetime64[ns]
TS_DIFF_TD     timedelta64[ns]
TS_DIFF秒                 int32
TS_DIFF分               float64
TS_DIFF時               float64
Length: 12, dtype: object


,TS,Power,Temperature,Pressure,Uptime,Status,Outcome,REF_TS,TS_DIFF_TD,TS_DIFF秒,TS_DIFF分,TS_DIFF時
0,2021-04-01 03:10:00,1079,308,0,3,0,202,2021-04-01 03:11:00,-1 days +23:59:00,-60,-1.0,-0.016667
1,2021-04-01 03:11:00,1083,308,0,3,0,202,2021-04-01 03:11:00,00:00:00,0,0.0,0.000000
2,2021-04-01 03:12:00,1080,309,0,3,0,202,2021-04-01 03:11:00,00:01:00,60,1.0,0.016667
3,2021-04-01 03:13:00,1080,309,0,3,0,202,2021-04-01 03:11:00,00:02:00,120,2.0,0.033333


# ⑥ タイムスタンプの大小比較
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/a04e3726-de0a-b0c6-ed92-5b4fe1f6a4ea.png)


In [9]:
df5=df.copy()
#文字列から日付型データ生成
df5['REF_TS']=pd.to_datetime('2021-04-01 03:11:00')

df5["AFTER_REF"]=(df5["TS"]>df5["REF_TS"])
print(df5.dtypes)
df5

TS             datetime64[ns]
Power                   int64
Temperature             int64
Pressure                int64
Uptime                  int64
Status                  int64
Outcome                 int64
REF_TS         datetime64[ns]
AFTER_REF                bool
dtype: object


,TS,Power,Temperature,Pressure,Uptime,Status,Outcome,REF_TS,AFTER_REF
0,2021-04-01 03:10:00,1079,308,0,3,0,202,2021-04-01 03:11:00,False
1,2021-04-01 03:11:00,1083,308,0,3,0,202,2021-04-01 03:11:00,False
2,2021-04-01 03:12:00,1080,309,0,3,0,202,2021-04-01 03:11:00,True
3,2021-04-01 03:13:00,1080,309,0,3,0,202,2021-04-01 03:11:00,True


# ⑦ タイムスタンプの加算減算
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/2f25e542-fdb3-2389-254e-d32b20aff0dd.png)


In [10]:
import datetime
df5=df.copy()

df5['3H_AFTER']=df5["TS"]+ datetime.timedelta(hours=3)
#以下でも実現は可能
#df5['3H_AFTER']=df5["TS"]+ np.timedelta64(3,'h')
print(df5.dtypes)
df5

TS             datetime64[ns]
Power                   int64
Temperature             int64
Pressure                int64
Uptime                  int64
Status                  int64
Outcome                 int64
3H_AFTER       datetime64[ns]
dtype: object


,TS,Power,Temperature,Pressure,Uptime,Status,Outcome,3H_AFTER
0,2021-04-01 03:10:00,1079,308,0,3,0,202,2021-04-01 06:10:00
1,2021-04-01 03:11:00,1083,308,0,3,0,202,2021-04-01 06:11:00
2,2021-04-01 03:12:00,1080,309,0,3,0,202,2021-04-01 06:12:00
3,2021-04-01 03:13:00,1080,309,0,3,0,202,2021-04-01 06:13:00


In [11]:
import sys

print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [12]:
print(pd.__version__)

1.0.5


In [13]:
print(np.__version__)

1.19.2
